In [34]:
import requests
from bs4 import BeautifulSoup
import time
import random
import csv

# Definimos un header con el User-Agent de una Mac para evitar baneos
user_agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1'
]

# Función para obtener headers aleatorios
def obtener_headers():
    headers = {
        'User-Agent': random.choice(user_agents)
    }
    return headers


# Función para obtener los títulos y los enlaces de las películas en cada página
def obtener_enlaces_peliculas(url, intentos=0):
    max_reintentos = 5
    try:
        response = requests.get(url, headers=obtener_headers())
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            movie_links = []
            movie_titles = soup.find_all('div', class_='movie-title')

            if not movie_titles:
                # Si no se encontraron películas, intentar hasta 5 veces
                print(f"[DEBUG] No se encontraron películas en la URL: {url} - Reintento {intentos+1}/{max_reintentos}")
                if intentos < max_reintentos:
                    time.sleep(2)  # Esperar un momento antes de reintentar
                    return obtener_enlaces_peliculas(url, intentos + 1)
                else:
                    print(f"[ERROR] No se encontraron películas después de {max_reintentos} reintentos.")
                    return []

            for title in movie_titles:
                a_tag = title.find('a')
                if a_tag:
                    enlace = a_tag.get('href')
                    nombre = a_tag.get('title').strip()
                    if not enlace.startswith('https'):
                        enlace = f"https://www.filmaffinity.com{enlace}"
                    movie_links.append({'nombre': nombre, 'enlace': enlace})
            return movie_links
        else:
            print(f"[ERROR] Status Code: {response.status_code}")
            return []
    except Exception as e:
        print(f"[ERROR] Ocurrió un error al obtener los enlaces de películas: {str(e)}")
        return []


# Función para obtener el enlace a las críticas de una película dentro de 'movie-reviews-box'
def obtener_criticas(url_pelicula):
    try:
        print(f"[INFO] Accediendo al enlace de la película: {url_pelicula}")
        response = requests.get(url_pelicula, headers=obtener_headers())
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Buscar el div con id 'movie-reviews-box'
            reviews_box = soup.find('div', id='movie-reviews-box')
            if reviews_box:
                a_tag = reviews_box.find('a', text=lambda t: 'críticas' in t.lower())
                if a_tag:
                    href_criticas = a_tag.get('href')
                    numero_criticas = a_tag.get_text(strip=True).split()[0]  # Obtener el número de críticas

                    # Construir el enlace completo si es relativo
                    if not href_criticas.startswith('https'):
                        href_criticas = f"https://www.filmaffinity.com{href_criticas}"

                    print(f"[DEBUG] Enlace a críticas encontrado: {href_criticas}")
                    print(f"[DEBUG] Número de críticas: {numero_criticas}")

                    return {
                        'enlace_criticas': href_criticas,
                        'numero_criticas': numero_criticas
                    }
            else:
                print(f"[WARNING] No se encontró 'movie-reviews-box' en la película: {url_pelicula}")
                return None
        else:
            print(f"[ERROR] No se pudo obtener la página de la película, código de estado: {response.status_code}")
            return None
    except Exception as e:
        print(f"[ERROR] Ocurrió un error al obtener las críticas: {str(e)}")
        return None


# Función para obtener detalles de las críticas desde el enlace de críticas
def obtener_detalles_criticas(url_criticas, intentos_max=3):
    try:
        criticas = []
        base_url, review_id = url_criticas.rsplit('/', 1)  # Separar el ID de la crítica final
        base_url = base_url.rsplit('/', 1)[0]  # Separar el número de la página de la crítica
        for intento in range(1, intentos_max + 1):  # Iterar cambiando solo el número de página
            new_url_criticas = f"{base_url}/{intento}/{review_id}"
            print(f"[INFO] Accediendo al enlace de críticas página {intento}: {new_url_criticas}")

            response = requests.get(new_url_criticas, headers=obtener_headers())
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                # Buscar el div con clase 'reviews-wrapper' que contiene la lista de críticas
                reviews_wrapper = soup.find('div', class_='reviews-wrapper')
                if reviews_wrapper:
                    # Iterar sobre cada crítica individual
                    reviews = reviews_wrapper.find_all('div', class_='fa-shadow movie-review-wrapper rw-item')
                    print(f"[INFO] {len(reviews)} críticas encontradas en la página {intento}.")
                    for review in reviews:
                        usuario = review.find('div', class_='mr-user-nick').get_text(strip=True)
                        puntuacion = review.find('div', class_='user-reviews-movie-rating').get_text(strip=True)
                        texto_critica = review.find('div', class_='review-text1').get_text(strip=True)

                        # Añadir crítica a la lista
                        criticas.append({
                            'usuario': usuario,
                            'puntuacion': puntuacion,
                            'texto': texto_critica
                        })

                        # Mostrar depuración de la crítica
                        print(f"[DEBUG] Usuario: {usuario}, Puntuación: {puntuacion}")
                        print(f"[DEBUG] Texto de la crítica: {texto_critica[:100]}...")
            else:
                print(f"[ERROR] No se pudo acceder al enlace de críticas, código de estado: {response.status_code}")
                break  # Salir del bucle si no se pudo acceder

        return criticas
    except Exception as e:
        print(f"[ERROR] Ocurrió un error al obtener los detalles de las críticas: {str(e)}")
        return []


# URL base para las películas de Netflix
url_base = "https://www.filmaffinity.com/es/category.php?id=new_netflix&page="

# Inicializamos variables
pagina = 1
peliculas_totales = []

# Crear archivo CSV y escribir encabezados
csv_filename = 'peliculas_y_criticas.csv'
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Encabezados
    writer.writerow(['Película', 'Enlace Película', 'Enlace Críticas', 'Número Críticas', 'Usuario', 'Puntuación', 'Texto Crítica'])

# Bucle para iterar sobre la paginación de las películas
while True:
    try:
        # Construimos la URL con el número de página
        url = url_base + str(pagina)
        print(f"[INFO] Scrapeando la página {pagina}...")
        print(f"[DEBUG] Enlace de la página: {url}")  # Mostrar el enlace de la página que se está scrapeando

        # Obtenemos los enlaces de las películas en la página actual
        enlaces_peliculas = obtener_enlaces_peliculas(url)

        # Si no hay más películas, salimos del bucle
        if not enlaces_peliculas:
            print("[INFO] No hay más películas.")
            break

        # Procesar cada película encontrada
        for pelicula in enlaces_peliculas:
            print(f"[INFO] Procesando película: {pelicula['nombre']} - Enlace: {pelicula['enlace']}")

            # Obtenemos las críticas de la película
            criticas_info = obtener_criticas(pelicula['enlace'])
            if criticas_info:
                pelicula['enlace_criticas'] = criticas_info['enlace_criticas']
                pelicula['numero_criticas'] = criticas_info['numero_criticas']

                # Obtener detalles de las críticas desde el enlace, intentando en 3 páginas
                detalles_criticas = obtener_detalles_criticas(criticas_info['enlace_criticas'])
                pelicula['detalles_criticas'] = detalles_criticas

                # Escribir los datos en el archivo CSV
                with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    for critica in detalles_criticas:
                        writer.writerow([
                            pelicula['nombre'],
                            pelicula['enlace'],
                            pelicula['enlace_criticas'],
                            pelicula['numero_criticas'],
                            critica['usuario'],
                            critica['puntuacion'],
                            critica['texto']
                        ])
            else:
                pelicula['enlace_criticas'] = None
                pelicula['numero_criticas'] = '0'
                pelicula['detalles_criticas'] = []

        # Pasamos a la siguiente página
        pagina += 1

        # Añadimos un pequeño retraso para no sobrecargar el servidor
        time.sleep(1)

    except Exception as e:
        print(f"[ERROR] Ocurrió un error en el bucle principal: {str(e)}")
        break

print(f"[INFO] Proceso completado. Datos guardados en {csv_filename}")


[INFO] Scrapeando la página 1...
[DEBUG] Enlace de la página: https://www.filmaffinity.com/es/category.php?id=new_netflix&page=1
[INFO] Procesando película: Rebel Ridge - Enlace: https://www.filmaffinity.com/es/film782882.html
[INFO] Accediendo al enlace de la película: https://www.filmaffinity.com/es/film782882.html


/tmp/ipykernel_5575/429220776.py:72: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  a_tag = reviews_box.find('a', text=lambda t: 'críticas' in t.lower())


[DEBUG] Enlace a críticas encontrado: https://www.filmaffinity.com/es/reviews/1/782882.html
[DEBUG] Número de críticas: críticas
[INFO] Accediendo al enlace de críticas página 1: https://www.filmaffinity.com/es/reviews/1/782882.html
[INFO] 3 críticas encontradas en la página 1.
[DEBUG] Usuario: nigeru, Puntuación: 6
[DEBUG] Texto de la crítica: Vista Rebel Ridge 2024. Un producto de Netflix de 123 minutos de metraje real, para mi gusto si tuvi...
[DEBUG] Usuario: AFICIONADO, Puntuación: 4
[DEBUG] Texto de la crítica: Denunciar el racismo y la corrupción policial no es suficiente para justificar una mala película , a...
[DEBUG] Usuario: Raúl, Puntuación: 6
[DEBUG] Texto de la crítica: Rebel Ridge (2024).“Lo más gracioso es que hasta anteayer nunca había estado en una pelea callejera....
[INFO] Accediendo al enlace de críticas página 2: https://www.filmaffinity.com/es/reviews/2/782882.html
[ERROR] No se pudo acceder al enlace de críticas, código de estado: 404
[INFO] Procesando película: